In [1]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

#### - Get every serie paths and generate JSON file 

In [3]:
series_paths = get_series_path('/media/deeplearning/VERBATIM HD/LNH073B_Validated_DICOM')
export_folder = '/media/deeplearning/VERBATIM HD/LNH073B_JSON'

In [4]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
json_path = export_folder
list_dir_json = os.listdir(json_path)
list_json = []
for file_ in list_dir_json : 
    list_json.append([os.path.join(json_path, file_)])

#### - Get informations from every json in list 

In [18]:
for liste in list_json : 
    #print(list_json.index(liste))
    with open(liste[0]) as json_file : 
        reader = json.load(json_file)

        #patient_id = reader["patient"]["PatientID"]
        patient_id = reader['path'].split('/')[5]
        study_uid = reader["study"]['StudyInstanceUID']
        modal = reader['series']['Modality']

        path = reader['path']

        #type_ = reader['study']["AccessionNumber"]
        #print(type_)
        date = reader['study']['StudyDate']
        #name = '_'.join(reader['patient']['PatientName'].split('^')).lower()
        liste.append(path)
        liste.append(modal)
        #liste.append(type_)
        liste.append(patient_id)
        #liste.append(name)
        liste.append(study_uid)
        liste.append(date)

In [19]:
list_json[0]

['/media/deeplearning/VERBATIM HD/LNH073B_JSON/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1722.json',
 '/media/deeplearning/VERBATIM HD/LNH073B_Validated_DICOM/0/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1721/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1722/7030105081006150309',
 'CT',
 '0',
 '1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1721',
 '20081006']

#### - Keep only pet0/pet screening 

In [ ]:
not_pet_screening = []
for data in list_json : 
    #print(data[3])
    if data[3] != 'PET_Screening' : 
        not_pet_screening.append(data)
print(len(not_pet_screening))
for serie in not_pet_screening : 
    list_json.remove(serie)

In [11]:
print("Nbr de serie pet0/screening :", len(list_json))

Nbr de serie pet0/screening : 1199


#### - Reunite CT/PT series with same study uid 

In [20]:
#get every study_uid 
liste = []
for data in list_json: 
    liste.append(data[-2])
#print(liste)
study_uid = []
study_uid.append(liste[0])
for uid in liste : 
    if uid not in study_uid : 
       study_uid.append(uid)

In [21]:
print('Nombre de study :', len(study_uid))

Nombre de study : 600


In [22]:
#find serie for each study uid 
data = []
for uid in study_uid : 
    #print(uid)
    sub = []
    for serie in list_json : 
        if uid in serie: 
            #print(serie)
            sub.append(serie)
    data.append(sub)


In [23]:
dataset = []
for serie in data : 
    #print(serie)
    if len(serie) != 1 : 
        serie[0].append(serie[1][0])
        serie[0].append(serie[1][1])
        serie[0].append(serie[1][2])

    dataset.append(serie[0])

In [24]:
dataset[0]

['/media/deeplearning/VERBATIM HD/LNH073B_JSON/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1722.json',
 '/media/deeplearning/VERBATIM HD/LNH073B_Validated_DICOM/0/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1721/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1722/7030105081006150309',
 'CT',
 '0',
 '1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1721',
 '20081006',
 '/media/deeplearning/VERBATIM HD/LNH073B_JSON/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298207.2073.json',
 '/media/deeplearning/VERBATIM HD/LNH073B_Validated_DICOM/0/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298194.1721/1.2.276.0.7230010.3.1.4.1136891754.1092.1223298207.2073/7030105081006150327',
 'PT']

In [ ]:
#### Clean dataset : if there is no json, double study_uid, if not CT AND PT

In [25]:
#no json 
i = 0
no_json_liste = []
for data in dataset : 
    if len(data) != 9 : 
        i += 1 
        no_json_liste.append(data)

if i != 0 : 
    print("number of study with no json : {}".format(i))

    for no_json_file in no_json_liste : 
        no_json_liste.remove(no_json_file)

    print("dataset cleaned ")

number of study with no json : 1
dataset cleaned 


In [26]:
#double study_uid 
double = []
double.append(dataset[0])
for data in dataset : 
    if data not in double : 
        double.append(data)

if len(double) == len(dataset) : 
    print('No double study uid, dataset cleaned')

No double study uid, dataset cleaned


In [27]:
#check Modality 
index = []
for data in dataset : 
    modal = []
    modal.append(data[2])
    modal.append(data[-1])
    #print(modal)
    if modal[0] == modal[1] : 
        index.append(dataset.index(data))
#print(index)
#if index is not empty : 
if len(index) != 0 :
    liste_to_remove = []
    for ind in index : 
        liste_to_remove.append(dataset[ind])

    for r in liste_to_remove : 
        dataset.remove(r)

print("dataset cleaned")

dataset cleaned


In [28]:
print("Nombre de study après clean :", len(dataset))

Nombre de study après clean : 588


#### - Sorted studies from 1 patient if necessary 

In [29]:
liste_patient_id = []
for data in dataset : 
    if data[3] not in liste_patient_id :
        liste_patient_id.append(data[3])

print("Nombre de patients : ", len(liste_patient_id))

Nombre de patients :  228


In [41]:
liste_pet_to_sort = []
for patient_id in liste_patient_id : 
    liste = []
    for data in dataset : 
        if patient_id == data[3] : 
            liste.append(data)

    liste_pet_to_sort.append(liste)

In [42]:
len(liste_pet_to_sort)

228

In [43]:
from datetime import datetime 
import numpy as np
for patient in liste_pet_to_sort : 
    if len(patient) != 1 : 
        #liste_date = []
        for i in range(len(patient)):
            #liste_date.append(datetime.strptime(patient[i][5], "%Y%m%d"))
            patient[i][5] = datetime.strptime(patient[i][5], "%Y%m%d")
        #print(patient)
        patient.sort(key=lambda patient:patient[5])
        #print("date :", liste_date)
        #new_liste_date = sorted(liste_date)
        #print("date sorted :", new_liste_date)




TypeError: strptime() argument 1 must be str, not datetime.datetime

In [ ]:
from datetime import datetime
remove = []
for patient in serie : 
    print(serie.index(patient))
    date_liste = []
    for study in patient:
        with open(study[0]) as json_file : 
            reader = json.load(json_file)
            #print(reader)
            date = reader['series']['AcquisitionDate']
            date_liste.append(date)
    print(date_liste)
    if (datetime.strptime(date_liste[0], "%Y%m%d")) < (datetime.strptime(date_liste[1], "%Y%m%d")) : 
        print("1ere")
        remove.append(patient[1])
    else : 
        print("2ieme")
        remove.append(patient[0])
    

    

print(remove)
for r in remove :
    dataset.remove(r)

#### - Prepare json file to generate nifti in "nifti_builder_from_json.ipynb" script

In [ ]:
study_results = []
for liste in dataset : 
    subliste = []
    subliste.append(liste[1]) #path_1
    subliste.append(liste[2]) #modal 1
    subliste.append(liste[-2]) #path_2
    subliste.append(liste[-1]) #modal_2
    subliste.append(liste[6]) #study_uid
    subliste.append(liste[3]) #type
    subliste.append(liste[4]) #patient id
    subliste.append(liste[5]) #patient name

    study_results.append(subliste)

In [ ]:
study_results[0]

In [ ]:
#save results as json 
directory = '/media/deeplearning/LACIE SHARE'
filename = 'relevance_dataset'
write_json_file(directory, filename, study_results)